In [1]:
# preliminaries

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from sktime.classification.interval_based import TimeSeriesForestClassifier



ModuleNotFoundError: No module named 'matplotlib'

In [2]:
import numpy as np
import pandas as pd

# load data and fix issues w original

weather = pd.read_csv('madweather.csv')
del weather['Unnamed: 0']
weather.head()
weather.dtypes

DATE     object
PRCP    float64
SNOW    float64
TAVG    float64
TMAX    float64
TMIN    float64
dtype: object

In [27]:
# change data from object to datetime
weather['DATE'] = pd.to_datetime(weather.DATE, format = '%m/%Y')
data = weather.drop(['DATE'], axis = 1)
data.index = pd.DatetimeIndex(weather.index).to_period('M')

In [34]:
# missing values
cols = data.columns
for j in cols:
    for i in range(0, len(data)):
        if data[j][i] == -200:
            data[j][i] = data[j][i-1]

data = data.replace(np.inf, np.nan).replace(-np.inf, np.nan).dropna()

In [32]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [35]:
data.head()


,PRCP,SNOW,TAVG,TMAX,TMIN
1970-01,0.69,4.5,23.7,30.8,16.6
1970-01,1.26,9.1,13.0,21.5,4.5
1970-01,1.33,12.5,25.1,32.3,17.9
1970-01,1.31,5.0,48.0,57.8,38.3
1970-01,1.87,0.0,60.5,70.9,50.2


In [36]:
# split into test and train
train = data[:int(0.8*(len(data)))]
test = data[int(0.8*(len(data))):]


In [26]:
from statsmodels.tsa.vector_ar.var_model import VAR

In [37]:
model = VAR(endog=train)
model_fit = model.fit(4)

In [38]:
prediction = model_fit.forecast(model_fit.y, steps = len(test))

C:\Users\nicol\AppData\Local\R-MINI~1\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [40]:
# dataframe of predictions
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,5):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

In [53]:
# check rmse
from sklearn.metrics import mean_squared_error
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(test[i], pred[i])))

rmse value for PRCP is :  2.39785737737556
rmse value for SNOW is :  7.541906508517674
rmse value for TAVG is :  19.29496956298127
rmse value for TMAX is :  20.301703321281227
rmse value for TMIN is :  18.384211727630934


In [55]:
#final predictions
modelfin = VAR(endog=data)
modelfin_fit = modelfin.fit()
ypred = modelfin_fit.forecast(modelfin_fit.y, steps = 1)
print(ypred)

[[ 1.92191634  7.479183   29.41469217 37.60364039 21.22414481]]


C:\Users\nicol\AppData\Local\R-MINI~1\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)
